In [1]:
# uncomment the code below to install the autograd package
# !pip install autograd

In [2]:
# uncomment the code below to install the pymanopt package
# !pip install pymanopt

In [3]:
import numpy as np
import pandas as pd
import scipy.io as sio
import scipy.linalg as la
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import scale,LabelEncoder

In [4]:
import numpy as np
import pymanopt as pm
import scipy.linalg as la
import autograd.numpy as anp
from pymanopt.optimizers import SteepestDescent
import autograd.numpy.linalg as ala
import pymanopt.manifolds as manifolds
from sklearn.preprocessing import scale
from sklearn.metrics import pairwise_distances

class JDIP:

    def __init__(self,dimension=30,lamda=1e-5,sigma=None,maxiter=5,verbosity=0):
        """

        """        
        args_values = locals()
        args_values.pop("self")
        for arg,value in args_values.items():
            setattr(self,arg,value)
    
    def fit(self,Xs,ys,Xt,yt,Ws=None,Wt=None):
        """
        
        """
        ns,dim = Xs.shape
        nt = Xt.shape[0]
        Xs,Xt = scale(Xs,with_std=False),scale(Xt,with_std=False)
        
        y = np.hstack((ys,yt))
        Ky = np.array(y[:,None]==y,dtype=np.float64)   
 
        if self.sigma is None:
            pairwise_dist = pairwise_distances(Xs,Xt,'sqeuclidean')
            self.sigma = np.sqrt(np.median(pairwise_dist[pairwise_dist!=0]) / 2.0)
        
        
        manifold = manifolds.Product([manifolds.Stiefel(dim, self.dimension),manifolds.Stiefel(dim, self.dimension)])
        @pm.function.autograd(manifold)
        # construct objective function 
        def objW(Ws,Wt):
            """
            code from 
            https://stackoverflow.com/questions/47271662/what-is-the-fastest-way-to-compute-an-rbf-kernel-in-python
            """                                   
            WX = anp.vstack((anp.dot(Xs,Ws),anp.dot(Xt,Wt)))
            WX_norm = anp.sum(WX ** 2, axis = -1)
            K_W = WX_norm[:,None] + WX_norm[None,:] - 2 * anp.dot(WX, WX.T)
                
            H = np.power(np.pi * self.sigma**2,self.dimension / 2.0) * anp.exp(-K_W / (4 * self.sigma**2)) * Ky
            Temp = anp.exp(-K_W / (2 * self.sigma**2)) * Ky
            h = anp.mean(Temp[:ns],axis=0) - anp.mean(Temp[ns:],axis=0)
            
            theta = ala.solve(H + self.lamda * anp.eye(ns + nt),h)
            D = anp.dot(h,theta)                       
            return D
        
        problem = pm.Problem(manifold=manifold, cost=objW)
        optimizer = SteepestDescent(max_iterations=self.maxiter,verbosity=self.verbosity)
        
        if Ws is None and Wt is None:
            W = optimizer.run(problem).point 
        else:
            W = optimizer.run(problem,initial_point=[Ws[:,:self.dimension],Wt[:,:self.dimension]]).point   
        return W


In [5]:
def readData(sc,tg): 
    data = sio.loadmat('OC10_vgg6/' + sc + '.mat')# source domain 
    Xs,ys = data['FTS'].astype(np.float64),data['LABELS'].ravel()
    ys = LabelEncoder().fit(ys).transform(ys).astype(np.float64)

    data = sio.loadmat('OC10_vgg6/' + tg + '.mat')# target domain 
    Xt,yt = data['FTS'].astype(np.float64),data['LABELS'].ravel()
    yt = LabelEncoder().fit(yt).transform(yt).astype(np.float64)
    
    Xs = Xs / la.norm(Xs,axis=1,keepdims=True)
    Xt = Xt / la.norm(Xt,axis=1,keepdims=True)
    return Xs,ys,Xt,yt

In [6]:
estimator = LinearSVC()

In [7]:
def source_partition(Xs,ys,samp_per_class,seed):
    Xs_tr,ys_tr = [],[]
    ys = ys.astype(int)
    for i in np.unique(ys):
        Xi,yi = Xs[ys==i],ys[ys==i]
        np.random.seed(seed)
        s = np.random.choice(len(yi),samp_per_class,replace=False)
        Xi_tr,yi_tr = Xi[s].tolist(),yi[s].tolist()
        Xs_tr.extend(Xi_tr),ys_tr.extend(yi_tr)

    return np.array(Xs_tr),np.array(ys_tr)

def target_partition(Xt,yt,samp_per_class,seed):
    Xt_tr,yt_tr,Xt_te,yt_te = [],[],[],[]
    yt = yt.astype(int)
    for i in np.unique(yt):
        Xi,yi = Xt[yt==i],yt[yt==i]
        np.random.seed(seed) 
        s1 = np.random.choice(len(yi),samp_per_class,replace=False)
        s2 = list(set(range(len(yi))) - set(s1))    
        Xi_tr,yi_tr,Xi_te,yi_te = Xi[s1].tolist(),yi[s1].tolist(),Xi[s2].tolist(),yi[s2].tolist()
        Xt_tr.extend(Xi_tr),yt_tr.extend(yi_tr),Xt_te.extend(Xi_te),yt_te.extend(yi_te)

    return np.array(Xt_tr),np.array(yt_tr),np.array(Xt_te),np.array(yt_te)

In [8]:
Xs,ys,Xt,yt = readData('dslr','webcam')

souSamp_per_class = 8
tarSamp_per_class = 3

Xs_sub,ys_sub = source_partition(Xs,ys,souSamp_per_class,0)
Xtl,ytl,Xtu,ytu = target_partition(Xt,yt,tarSamp_per_class,0)
                
ytu0 = estimator.fit(np.vstack((Xs_sub,Xtl)),np.hstack((ys_sub,ytl))).predict(Xtu)
W0 = PCA().fit(np.vstack((Xs_sub,Xt))).components_.T
Ws,Wt = W0,W0
for k in range(10):
    Xt_,yt0_ = np.vstack((Xtl,Xtu)),np.hstack((ytl,ytu0))
    W = JDIP(dimension=100,lamda=1e-5,sigma=.5,maxiter=5,verbosity=0).fit(Xs_sub,ys_sub,Xt_,yt0_,Ws,Wt)
    Xs_sub_r,Xtl_r,Xtu_r = Xs_sub.dot(W[0]),Xtl.dot(W[1]),Xtu.dot(W[1])
        
    Xs_sub_r = Xs_sub_r / la.norm(Xs_sub_r,axis=1,keepdims=True)
    Xtl_r = Xtl_r / la.norm(Xtl_r,axis=1,keepdims=True)
    Xtu_r = Xtu_r / la.norm(Xtu_r,axis=1,keepdims=True)
                
    ytu0 = estimator.fit(np.vstack((Xs_sub_r,Xtl_r)),np.hstack((ys_sub,ytl))).predict(Xtu_r)
    #print(k, '-th iter.', estimator.score(Xtu_r,ytu))
                                
print(estimator.fit(np.vstack((Xs_sub_r,Xtl_r)),np.hstack((ys_sub,ytl))).score(Xtu_r,ytu) * 100.)

98.86792452830188
